# LSTM Model to generate clean lyrics 
Stefel Smith, Cleo De Rocco , Abigail Kayser 
COSC072 Final Project
Code adapted from CHatGPT prompt 

Here is a link to how LSTM models work: https://www.youtube.com/watch?v=YCzL96nL7j0 , https://levelup.gitconnected.com/lyrics-generation-using-lstm-5a5a0bcac4fa 


The LSTM Models can perfom well with relatively small datastes abd is suitabble for generating shoter bodies of text.

In [568]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [569]:
# read in the data set 
train_df = pd.read_csv("t5lyricdata.csv")
explicit = train_df["kbLyric"].isnull().sum()
clean = train_df["ogLyric"].isnull().sum()
print(explicit,clean)
train_df

0 0


,ogArtist,songName,badword,count,category,year,isCensored,isPresent,ogLyric,kbLyric
0,Ariana Grande,7 rings,bitch,2.0,profanity,2019.0,1.0,1.0,Bought matching diamonds for six of my bitches,Bought matching diamonds so they know what thi...
1,Britney Spears,till the world ends,blow,2.0,alcohol & drugs,2011.0,1.0,1.0,"Baby, let me blow your mind tonight","Baby, come on wave your hands tonight"
2,Britney Spears,till the world ends,touch,1.0,sexual,2011.0,1.0,1.0,"Get you off with the touch, dancing in the dark",Set it off with a track dancing in the dark
3,Elle King,ex's and oh's,high,1.0,alcohol & drugs,2016.0,1.0,1.0,"I get high, and I love to get low","I get by, and I love to lay low"
4,Elle King,ex's and oh's,man,1.0,identity,2016.0,1.0,1.0,"Well, I had me a boy, turned him into a man","Well, I knew me a boy, and yes he was a friend"
...,...,...,...,...,...,...,...,...,...,...
770,Kesha,Tik Tok,NaN,NaN,NaN,NaN,NaN,NaN,"Before I leave, brush my teeth with a bottle o...","Before I leave, brush my teeth and then I go a..."
771,Kesha,Tik Tok,NaN,NaN,NaN,NaN,NaN,NaN,Trying to get a little bit tipsy,Trying to get a little bit silly
772,Kesha,Tik Tok,NaN,NaN,NaN,NaN,NaN,NaN,"Tonight, I'ma fight 'til we see the sunlight","Tonight, I'm alright 'til we see the sunlight"
773,Rudimental,these days,NaN,NaN,NaN,NaN,NaN,NaN,Cigarettes in the ash tray,I get lost in the memories


In [570]:
explicit_lyrics = train_df["kbLyric"].tolist()[:300]
clean_lyrics = train_df["ogLyric"].tolist()[:300]


In [571]:
def process_data(explicit_lyrics, clean_lyrics):
    explicit_prcoessed = [] 
    clean_processed = [] 

    for i in range(len(explicit_lyrics)):
        explicit_prcoessed.append(explicit_lyrics[i].lower())
        clean_processed.append(clean_lyrics[i].lower())

    return explicit_prcoessed, clean_processed

explicit_lyrics, clean_lyrics = process_data(explicit_lyrics,clean_lyrics)

In [572]:
# Tokenize lyrics and create a vocabulary
lyrics = explicit_lyrics + clean_lyrics
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics)
vocab_size = len(tokenizer.word_index) + 1

In [573]:
# Convert explicit and clean lyrics to sequences of word indices
explicit_sequences = tokenizer.texts_to_sequences(explicit_lyrics)
clean_sequences = tokenizer.texts_to_sequences(clean_lyrics)

In [574]:
# Pad sequences to have the same length
max_sequence_length = max(len(seq) for seq in explicit_sequences)
padded_explicit = pad_sequences(explicit_sequences, maxlen=max_sequence_length, padding='post')
padded_clean = pad_sequences(clean_sequences, maxlen=max_sequence_length, padding='post')

In [575]:

# Create input and target data
X = padded_explicit[:, :-1]
y = padded_clean[:, 1:]
y = to_categorical(y, num_classes=vocab_size)  # Convert y to one-hot encoding

# Reshape y to have a single dimension
y = np.argmax(y, axis=1)

In [576]:
X.shape, y.shape

((300, 14), (300, 936))

In [577]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_length-1))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(400))
model.add(Dense(vocab_size, activation='softmax'))
model.add(Dense(vocab_size, activation='softmax'))


# Compile the model with a higher learning rate
learning_rate = 0.015  # Set the desired learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

2023-05-29 14:30:43.193971: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 14:30:43.197480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 14:30:43.201178: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [578]:
# Train the model
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10


2023-05-29 14:30:44.218647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 14:30:44.222069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 14:30:44.225262: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 7s 145ms/step - loss: 227.2274
Epoch 2/10
10/10 [==============================] - 1s 134ms/step - loss: 220.3980
Epoch 3/10
10/10 [==============================] - 1s 132ms/step - loss: 214.7030
Epoch 4/10
10/10 [==============================] - 1s 135ms/step - loss: 209.5162
Epoch 5/10
10/10 [==============================] - 1s 147ms/step - loss: 204.9984
Epoch 6/10
10/10 [==============================] - 2s 164ms/step - loss: 201.3748
Epoch 7/10
10/10 [==============================] - 2s 179ms/step - loss: 198.0319
Epoch 8/10
10/10 [==============================] - 2s 153ms/step - loss: 195.0934
Epoch 9/10
10/10 [==============================] - 2s 161ms/step - loss: 192.3860
Epoch 10/10
10/10 [==============================] - 1s 149ms/step - loss: 189.8958


In [579]:
seed_text = "Bought diamond rings for six of my bitches"
generated_lyrics = []

# Generate 20 words (change as needed)
for _ in range(20):
    encoded_text = tokenizer.texts_to_sequences([seed_text])[0]
    encoded_text = np.array(encoded_text)
    encoded_text = pad_sequences([encoded_text], maxlen=max_sequence_length-1)
    y_pred = model.predict(encoded_text)

    # Sample a word from the predicted probabilities
    predicted_word_probs = y_pred[0][-1]  # Select probabilities of the last predicted word
    predicted_word_index = np.random.choice(predicted_word_probs.size)
    
    if predicted_word_index + 1 in tokenizer.index_word:
        next_word = tokenizer.index_word[predicted_word_index + 1]
    else:
        next_word = "<UNK>"  # Handle unknown words
    
    generated_lyrics.append(next_word)
    seed_text = " " + next_word

generated_lyrics = " ".join(generated_lyrics)
print(generated_lyrics)


2023-05-29 14:31:04.548991: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 14:31:04.551690: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 14:31:04.554225: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 31ms/step
i i i i i i i i i i i i i i i i i i i i
